<a href="https://colab.research.google.com/github/donw385/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/module2-join-datasets/LS_DS_122_Join_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science_

# Join datasets

Objectives
- concatenate data with pandas
- merge data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
  - Combine Data Sets: Standard Joins
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join

## Download data

We’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

In [0]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-01-15 20:39:58--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.19.11
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.19.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.2’

instacart_online_gr 100%[===================>] 196.03M  96.2MB/s    in 2.0s    

2019-01-15 20:40:00 (96.2 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.2’ saved [205548478/205548478]



In [0]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [0]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


## Goal: Reproduce this example

The first two orders for user id 1:

In [0]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

## Load data

Here's a list of all six CSV filenames

In [0]:
!ls -lh

total 877M
-rw-r--r-- 1  502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1  502 staff  270 May  2  2017 departments.csv
drwxr-xr-x 2  502 staff 4.0K May  2  2017 instacart_2017_05_01
-rw-r--r-- 1 root root  197M Oct 20  2017 instacart_online_grocery_shopping_2017_05_01.tar.gz
-rw-r--r-- 1  502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1  502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1  502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1  502 staff 2.1M May  2  2017 products.csv


For each CSV
- Load it with pandas
- Look at the dataframe's shape
- Look at its head (first rows)
- `display(example)`
- Which columns does it have in common with the example we want to reproduce?

### aisles

In [0]:
import pandas as pd
aisles = pd. read_csv('aisles.csv')

In [0]:
aisles.shape

(134, 2)

In [0]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [0]:
#aisle not useful
display(example)

### departments

In [0]:
departments = pd. read_csv('departments.csv')

In [0]:
departments.shape

(21, 2)

In [0]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [0]:
departments.sample(n=5)
#departments not useful

,department_id,department
6,7,beverages
12,13,pantry
11,12,meat seafood
7,8,pets
3,4,produce


### order_products__prior

In [0]:
order_products_prior = pd.read_csv('order_products__prior.csv')

In [0]:
order_products_prior.shape

(32434489, 4)

In [0]:
order_products_prior.head()

#order id, product id, add to cart order useful

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


### order_products__train

In [0]:
order_products_train = pd.read_csv('order_products__train.csv')

In [0]:
order_products_train.shape

(1384617, 4)

In [0]:
order_products_train.head()
#exact same columns as prior

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


### orders

In [0]:
orders = pd.read_csv('orders.csv')

In [0]:
orders.shape

(3421083, 7)

In [0]:
orders.head()
# need all except eval set, and days since prior order

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


### products

In [0]:
products = pd.read_csv('products.csv')

In [0]:
products.shape

(49688, 4)

In [0]:
products.head()
#product id, product name

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


## Concatenate order_products__prior and order_products__train

In [0]:
#append columns, add, axis=1, but better to add axis = column

order_products = pd.concat([order_products_prior,order_products_train])

order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [0]:
#validate working properly

assert len(order_products) == len(order_products_prior) + len(order_products_train)

In [0]:
#validate columns
assert len(order_products.columns) == len(order_products_prior.columns) == len(order_products_train.columns)

## Get a subset of orders — the first two orders for user id 1

In [0]:
condition = (orders.user_id==1) & (orders.order_number <= 2)
columns = ['user_id', 'order_id','order_number','order_dow','order_hour_of_day']
subset = orders.loc[condition,columns]
subset.head()

,user_id,order_id,order_number,order_dow,order_hour_of_day
0,1,2539329,1,2,8
1,1,2398795,2,3,7


## Merge dataframes

In [0]:
columns =['order_id','add_to_cart_order','product_id']
merged = pd.merge(subset, order_products[columns], how='inner', on='order_id')

In [0]:
subset.shape, merged.shape

((2, 5), (11, 7))

In [0]:
merged['product_name'] = products['product_name']
merged.head()

,user_id,order_id,order_number,order_dow,order_hour_of_day,add_to_cart_order,product_id,product_name
0,1,2539329,1,2,8,1,196,Chocolate Sandwich Cookies
1,1,2539329,1,2,8,2,14084,All-Seasons Salt
2,1,2539329,1,2,8,3,12427,Robust Golden Unsweetened Oolong Tea
3,1,2539329,1,2,8,4,26088,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,1,2539329,1,2,8,5,26405,Green Chile Anytime Sauce


In [0]:
final = pd.merge(merged,products[['product_id','product_name']])
final = final.sort_values(by=['order_number', 'add_to_cart_order'])

In [0]:
final.columns = [column.replace('_', ' ') 
                 for column in final]

In [0]:
final.head()

,user id,order id,order number,order dow,order hour of day,add to cart order,product id,product name


# Assignment

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

## Stretch challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.

In [0]:
total = pd.merge(order_products, products)


In [0]:
total.head()


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16


In [0]:
total.shape, order_products.shape, products.shape

((33819106, 7), (33819106, 4), (49688, 4))

In [0]:
product_counts = total['product_name'].value_counts().sort_index()

In [0]:
#answer to assignment
product_counts.sort_values(ascending=False).head(10)

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

In [0]:
total['order_hour_of_day'] = orders['order_hour_of_day']
total.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,order_hour_of_day
0,2,33120,1,1,Organic Egg Whites,86,16,8.0
1,26,33120,5,0,Organic Egg Whites,86,16,7.0
2,120,33120,13,0,Organic Egg Whites,86,16,12.0
3,327,33120,5,1,Organic Egg Whites,86,16,7.0
4,390,33120,28,1,Organic Egg Whites,86,16,15.0


In [0]:
total['order_hour_of_day'].min()
total['order_hour_of_day'].max()


23.0

In [0]:
order_hour_average= total.groupby('product_name')['order_hour_of_day'].mean()


In [118]:
order_hour_average.sort_values(ascending=False).head(10)

product_name
Honey/Lemon Cough Drops                                       14.768519
Sunrise Grilled Steak and Eggs Canine Cuisine Wet Dog Food    14.716216
French Lavender Hand Wash                                     14.414508
100% Juice No Added Sugar Orange Tangerine                    14.300000
Sweet & Salty Snack Mix                                       14.200000
Chocolate Caramel Cashew Trail Mix                            14.133333
Macaroni And Cheese                                           13.956092
Hair Bender Whole Bean Coffee                                 13.947747
Wafer, Chocolate                                              13.943182
G Series Lime Cucumber Sports Drink                           13.890351
Name: order_hour_of_day, dtype: float64

In [119]:
df_order=pd.DataFrame(order_hour_average)
df_order.head()

,order_hour_of_day
product_name,
#2 Coffee Filters,NaN
#2 Cone White Coffee Filters,NaN
#2 Mechanical Pencils,NaN
#4 Natural Brown Coffee Filters,NaN
& Go! Hazelnut Spread + Pretzel Sticks,NaN


In [120]:
order_hour_average.shape, product_counts.shape

((49685,), (49685,))

In [0]:
df=pd.DataFrame(product_counts)

In [126]:
df = df.rename(columns={'product_name':'orders'})
df.head()

,orders
#2 Coffee Filters,799
#2 Cone White Coffee Filters,14
#2 Mechanical Pencils,9
#4 Natural Brown Coffee Filters,247
& Go! Hazelnut Spread + Pretzel Sticks,174


In [0]:
df_merged=pd.concat([df,df_order],axis=1)

In [130]:
df_merged = df_merged.rename(columns={'order_hour_of_day':'order time'})

df_merged.head()


,orders,order time
product_name,,
#2 Coffee Filters,799,NaN
#2 Cone White Coffee Filters,14,NaN
#2 Mechanical Pencils,9,NaN
#4 Natural Brown Coffee Filters,247,NaN
& Go! Hazelnut Spread + Pretzel Sticks,174,NaN


In [132]:
df_merged.sort_values(by=['order time'], ascending=False)

,orders,order time
product_name,,
Honey/Lemon Cough Drops,108,14.768519
Sunrise Grilled Steak and Eggs Canine Cuisine Wet Dog Food,74,14.716216
French Lavender Hand Wash,193,14.414508
100% Juice No Added Sugar Orange Tangerine,30,14.300000
Sweet & Salty Snack Mix,35,14.200000
Chocolate Caramel Cashew Trail Mix,15,14.133333
Macaroni And Cheese,2733,13.956092
Hair Bender Whole Bean Coffee,2641,13.947747
"Wafer, Chocolate",88,13.943182


In [133]:
df_merged.head()

,orders,order time
product_name,,
#2 Coffee Filters,799,NaN
#2 Cone White Coffee Filters,14,NaN
#2 Mechanical Pencils,9,NaN
#4 Natural Brown Coffee Filters,247,NaN
& Go! Hazelnut Spread + Pretzel Sticks,174,NaN


In [134]:
condition_new = (df_merged.orders >= 2900)

df_merged_subset = df_merged.loc[condition_new]
df_merged_subset.shape


(2053, 2)

In [135]:
df_merged_subset = df_merged_subset.sort_values(by=['order time'], ascending=False)
df_merged_subset.head(24)

,orders,order time
product_name,,
Tiny Twists Pretzels,4732,13.661877
Boneless Beef Sirloin Steak,3220,13.638820
Mini Original Babybel Cheese,15941,13.588357
Natural Sharp Cheddar Sliced Cheese,7894,13.576514
Original Potato Chips,4139,13.565596
Distilled Water,16664,13.562590
Clementines,32194,13.548177
Philadelphia Original Cream Cheese,7535,13.528069
Total 2% with Strawberry Lowfat Greek Strained Yogurt,30866,13.519568
